## <center> Operating Room Scheduling-Assignment 1
#### <center>Author: Raffi Sahakyan
##### <center>Class: 19F-MGMTMSA403-1 Optimization
##### <center>Date: 15 Nov, 2019

<hr style="height:2px; border:none; color:#000; background-color:#000;">

In [245]:
#Importing Gurobi and other necessary functions
from gurobipy import *
import pandas as pd

#Defining Optimize and Update Function for Code Efficiency
def grb_optimize():
    mod.update()
    mod.optimize()

#### <center>Question a) - Part1 Optimization Model</center>

$$ {Objective\:Function} $$
<br>
$$\min_{i} \sum_{i=1}^{dep} \frac{S_i}{T_i}$$
<br>
$$ {Constraints} $$
<br>
$$s. t. \sum_{ijk=1}^{dep~room~day}=1$$
<br>
$$S_i\geq{0}$$
<br>
$$S_i\geq{} \sum_{jk=1}^{room~day} \sum_{ijk=1}^{dep~room~day}d_{jk} X_{ijk}-T_i$$
<br>
$$ {Decision\:Variables} $$
<br>
$$ S \: {Underallocation\:for\:department} $$
<br>
\begin{align}
\ x_{ijk} \in \{0,1\} \quad {i} = 1,\ldots,6 \quad {j} = 1,\ldots,5. \quad {k} = 1,\ldots,5 {\:3D\:Binary\:Matrix\:for\:location\:filling\:on\:dep\:,\:room\:,day}
\end{align}
<br>

#### <center>Question a) - Part 2 Python Code</center>

In [246]:
#Initializing the model
mod = Model()

#Variables
departments = 6 #i in the assignment
rooms = 5 #j in the assignment
days = 5 #k in the assignment

availability = [[9,9,9,9,7.5], [9,9,9,9,7.5],[9,9,9,9,7.5],[9,9,9,9,7.5],[9,8,8,8,6.5]] #Letter d in formulae
target = [213.5*.484,213.5*.042,213.5*.253,213.5*.074,213.5*.053,213.5*.095] #Letter T in formulae

# Define decision variables
S = mod.addVars(departments) #underallocation
X = mod.addVars(departments,rooms,days,vtype = GRB.BINARY) #3d binary matrix for the assignment of department for room in day

In [247]:
# Constraints
for room in range(rooms):
    for day in range(days):
        mod.addConstr(sum(X[department,room,day] for department in range(departments))==1)
        
for department in range(departments):
    mod.addConstr(S[department]>=target[department]-sum(availability[room][day]*X[department,room,day] for room in range(rooms) for day in range(days)))

for department in range(departments):
    mod.addConstr(S[department]>=0)        

#Setting Objective
mod.setObjective(sum(S[department]/target[department] for department in range(departments)), GRB.MINIMIZE)

#Calling optimize function
grb_optimize()

Optimize a model with 37 rows, 156 columns and 312 nonzeros
Variable types: 6 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 0.9575804
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 6 continuous, 150 integer (150 binary)

Root relaxation: objective 2.066116e-03, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00207    0    7    0.95758    0.00207   100%     -    0s
H    0     0                       0.3238502    0.00207  99.4%     -    0s
     0     0    0.02632    0   10    0.32385    0.02632  91.9%     -    0s
H    0     0                       0.2306614    0.02632  88.6%     -    

#### <center>Question a) - Part 3 Optimal Value Obtained</center>

In [248]:
print("Optimal value obtained at an optimal solution is",round(mod.objval,4))

Optimal value obtained at an optimal solution is 0.0519


#### <center>Question a) - Part 4 Schedule</center>

In [249]:
#Entries for schedule
Departments = ['General Surgery','Emergency','Neurosurgery','Opthamology','Oral Surgery','Otolaryngology']
Weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Rooms = ['Main-1','Main-2','Main-3','Main-4','Main-5']
schedule_df = pd.DataFrame(columns=Rooms,index=Weekdays)

#Defining Function for Schedule
def schedule_func():
    for index_d, department in enumerate(Departments):
        for index_r, room in enumerate(Rooms):
            for index_w, day in enumerate(Weekdays):
                if X[index_d,index_r,index_w].X == abs(1.0):
                    schedule_df[room][day] = department
    return (schedule_df)

schedule_func()

,Main-1,Main-2,Main-3,Main-4,Main-5
Monday,General Surgery,General Surgery,Neurosurgery,Neurosurgery,Neurosurgery
Tuesday,General Surgery,Emergency,Neurosurgery,Neurosurgery,Opthamology
Wednesday,Neurosurgery,General Surgery,General Surgery,General Surgery,Opthamology
Thursday,General Surgery,General Surgery,General Surgery,General Surgery,General Surgery
Friday,Oral Surgery,Otolaryngology,Oral Surgery,Otolaryngology,Otolaryngology


<hr style="height:2px; border:none; color:#000; background-color:#000;">

#### <center>Question b) - Part 1 Optimization Model</center>

$$ {Objective~Function} $$
<br>
$$\min_{i} \sum_{i=1}^{dep} \frac{S_i}{T_i}$$
<br>
$$ {Constraints} $$
<br>
$$s. t.\sum_{i,j,k=1}^{dep~room~day}=1$$
<br>
$$S_i\geq{0}$$
<br>
$$S_i\geq{} \sum_{j,k=1}^{room~day} \sum_{i,j,k=1}^{dep~room~day}d_{jk} X_{ijk}-T_i $$
<br>
$$\sum_{i,k=1}^{dep~day}{|}SC_{ik+1}-SC_{ik}{|} \leq{} 4 {\:Linearized\:in\:Code} $$ 
<br>
$$ {Decision~Variables} $$
<br>
$$ S \: {Underallocation\:for\:department} $$
<br>
\begin{align}
\ x_{ijk} \in \{0,1\} \quad {i} = 1,\ldots,6 \quad {j} = 1,\ldots,5 \quad {k} = 1,\ldots,5.
\end{align}
<br>
\begin{align}
\ SC_{ik} \in \{0,1\} \quad {i} = 1,\ldots,6 \quad {k} = 1,\ldots,5. {\:Switching\:Cost\:Variable}
\end{align}

#### <center>Question b) - Part 2 Python Code</center>

In [250]:
#Additional Decision Variable for part b
SC = mod.addVars(departments, days,vtype = GRB.BINARY) #SwithcingCost Variable

#Additional Constraints for part b - linearized
for department in range(departments):
    mod.addConstr(sum(SC[department, day] for day in range(day - 1)) <= 4)

for department in range(departments):
    mod.addConstr(sum(SC[department, day] for day in range(day - 1)) >= -4)

#Updating and Optimizing
grb_optimize()

Optimize a model with 49 rows, 186 columns and 348 nonzeros
Variable types: 6 continuous, 180 integer (180 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start produced solution with objective 0.051906 (0.01s)
Loaded MIP start with objective 0.051906

Presolve removed 18 rows and 30 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 6 continuous, 150 integer (150 binary)

Root relaxation: objective 2.066116e-03, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00207    0    7    0.05191    0.00207  96.0%     -    0s
     0     0    0.05148    0    3    0.05191    0.05148  0.81%     -    0s
     0     0    0.05177    0    2    0.05191    0.05177  0.26%     -    0s
     0     0 

#### <center>Question b) - Part 3 Optimal Value Obtained</center>

In [251]:
print("Optimal value obtained at an optimal solution is",round(mod.objval,4))

Optimal value obtained at an optimal solution is 0.0519


#### <center>Question b) - Part 4 Schedule</center>

In [252]:
schedule_func()

,Main-1,Main-2,Main-3,Main-4,Main-5
Monday,General Surgery,General Surgery,Neurosurgery,Neurosurgery,Neurosurgery
Tuesday,General Surgery,Emergency,Neurosurgery,Neurosurgery,Opthamology
Wednesday,Neurosurgery,General Surgery,General Surgery,General Surgery,Opthamology
Thursday,General Surgery,General Surgery,General Surgery,General Surgery,General Surgery
Friday,Oral Surgery,Otolaryngology,Oral Surgery,Otolaryngology,Otolaryngology


<hr style="height:2px; border:none; color:#000; background-color:#000;">

#### <center>Question c) - Part 1 Optimization Model</center>

$$ {Objective Function} $$
<br>
$$\min_{i} \sum_{i=1}^{dep} \frac{S_i}{T_i}$$
<br>
$$ {Constraints} $$
<br>
$$s. t.\sum_{i,j,k=1}^{dep~room~day}=1$$
<br>
$$S_i\geq{0}$$
<br>
$$S_i\geq{} \sum_{j,k=1}^{room~day} \sum_{i,j,k=1}^{dep~room~day}d_{jk} X_{ijk}-T_i$$
<br>
$$ X_{i,1,k} + X_{i,2,k} \leq{} {2 *} Z_{i,1,k} $$
<br>
$$ X_{i,3,k} + X_{i,4,k} \leq{} {2 *} Z_{i,2,k} $$
<br>
$$ X_{i,5,k}\leq{} Z_{i,3,k} $$ <br>
$$ \sum_{i,f,k=1}^{dep~floor~day} \leq{} 1 $$ 
$$ {Decision Variables} $$
<br>
$$ S \: {Underallocation\:for\:department} $$
<br>
\begin{align}
\ x_{ijk} \in \{0,1\} \quad {i} = 1,\ldots,6 \quad {j} = 1,\ldots,5 \quad {k} = 1,\ldots,5.
\end{align}
<br>
\begin{align}
\ Z_{ifk} \in \{0,1\} \quad {i} = 1,\ldots,6 \quad {f} = 1,\ldots,3 \quad {k} = 1,\ldots,5.
\end{align}

#### <center>Question c) - Part2 Python Code</center>

In [253]:
#Additional Variable for part c
floors=3

#Additional Decision Variable for part c
Z = mod.addVars(departments,floors,days,vtype = GRB.BINARY)

#Additional Constraints for part c
for department in range(departments):
    for day in range(days):
        mod.addConstr(X[department,0,day] + X[department,1,day] <= 2 * Z[department,0,day])

for department in range(departments):
    for day in range(days):
        mod.addConstr(X[department,2,day] + X[department,3,day] <= 2 * Z[department,1,day])
        
for department in range(departments):
    for day in range(days):
        mod.addConstr(X[department,4,day] <= Z[department,2,day])

for department in range(departments):
    for day in range(days):
        mod.addConstr(sum(Z[department,floor,day] for floor in range(floors))<=1)

#Updating and Optimizing
grb_optimize()

Optimize a model with 169 rows, 276 columns and 678 nonzeros
Variable types: 6 continuous, 270 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start did not produce a new incumbent solution

Found heuristic solution: objective 1.1082105
Presolve removed 78 rows and 90 columns
Presolve time: 0.00s
Presolved: 91 rows, 186 columns, 516 nonzeros
Variable types: 6 continuous, 180 integer (180 binary)

Root relaxation: objective 1.580699e-01, 72 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       0.1580699    0.00000   100%     -    0s
     0     0     cutoff    0         0.15807    0.15807  0.00%     -    0s

Explored 0 nodes (120 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 availab

#### <center>Question c) - Part 3 Optimal Value Obtained</center>

In [254]:
print("Optimal value obtained at an optimal solution is",round(mod.objval,4))

Optimal value obtained at an optimal solution is 0.1581


#### <center>Question c) - Part 4 Schedule</center>

In [255]:
schedule_func()

,Main-1,Main-2,Main-3,Main-4,Main-5
Monday,Neurosurgery,Neurosurgery,General Surgery,General Surgery,Otolaryngology
Tuesday,Neurosurgery,Neurosurgery,General Surgery,General Surgery,Otolaryngology
Wednesday,General Surgery,General Surgery,Oral Surgery,Otolaryngology,Neurosurgery
Thursday,Opthamology,Emergency,General Surgery,General Surgery,Neurosurgery
Friday,Opthamology,Neurosurgery,General Surgery,General Surgery,Oral Surgery
